# Text Classification using Naive Bayes

## Import Packages


In [12]:
import numpy as np
from sklearn.svm import SVC
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

## Dataset characteristics


In [13]:
categories = ['comp.graphics', 'sci.med']
twenty_train = fetch_20newsgroups(subset='train',
    categories=categories, shuffle=False, random_state=42)
twenty_test = fetch_20newsgroups(subset='test',
    categories=categories, shuffle=False, random_state=42)

print("Dataset properties on train section:")
print("\t Number of training data points: %d" % len(twenty_train.data))
print("\t Number of test data points: %d" % len(twenty_test.data))
print("\t Number of Classes: %d" % len(categories))
print("\t Class names: " ,(twenty_train.target_names))

Dataset properties on train section:
	 Number of training data points: 1178
	 Number of test data points: 785
	 Number of Classes: 2
	 Class names:  ['comp.graphics', 'sci.med']


## A sample of dataset

In [14]:
print("\n".join(twenty_train.data[0].split("\n")))

Subject: Why isolate it?
From: chinsz@eis.calstate.edu (Christopher Hinsz)
Organization: Calif State Univ/Electronic Information Services
Lines: 13

	Does anyone on this newsgroup happen to know WHY morphine was
first isolated from opium?  If you know why, or have an idea for where I
could look to find this info, please mail me.
	CSH
any suggestionas would be greatly appreciated

--
 "Kilimanjaro is a pretty tricky climb. Most of it's up, until you reach
the very, very top, and then it tends to slope away rather sharply."
					Sir George Head, OBE (JC)
------------------------------------------------------------------------------
LOGIC: "The point is frozen, the beast is dead, what is the difference?"
					Gavin Millarrrrrrrrrr (JC)



the category name of the instance can be found as follows:

In [15]:
print(twenty_train.target[0])
print(twenty_train.target_names[twenty_train.target[0]])

# print another different example
print(twenty_train.target[2])
print(twenty_train.target_names[twenty_train.target[2]])

1
sci.med
0
comp.graphics


some notes about the above two functions
* twenty_train.target[0]
    * input: index of data point
    * output 0 or 1
    * 0 --> sci.med class
    * 1 --> comp.graphics class

twenty_train.target_names[twenty_train.target[0]]
the name of the class that the data point belongs to

print the class names of the first 10 data examples

In [16]:
for t in twenty_train.target[:10]:
    print(twenty_train.target_names[t])

sci.med
sci.med
comp.graphics
sci.med
comp.graphics
sci.med
sci.med
comp.graphics
sci.med
sci.med


## Feature extraction


In [17]:
# bag of words
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_test_counts = count_vect.transform(twenty_test.data)

# Normalize
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_test_tf = tf_transformer.transform(X_test_counts)

print(X_train_tf.shape)
print(X_test_tf.shape)

(1178, 24614)
(785, 24614)


## Document classification using SVM

In [18]:
clf = SVC().fit(X_train_tf, twenty_train.target)

preds = clf.predict(X_test_tf)

accuracy = np.mean(preds == twenty_test.target)
print(f'Accuracy on test data is {accuracy*100}%.')

Accuracy on test data is 89.4267515923567%.


Try some real text

In [19]:
docs_new = ['OpenGL on the GPU is fast', 'Doctor takes care of patients']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, twenty_train.target_names[category]))

'OpenGL on the GPU is fast' => comp.graphics
'Doctor takes care of patients' => sci.med


## Naive Bayes with Bernoulli


In this task, we want to train a naive bayes classifier with bernoulli features.

As the first step, let us convert all features to binary values.

In [20]:
X_train_b = X_train_tf.sign()
X_test_b = X_test_tf.sign()
# if x == 0  --> 0
# if x > 0   --> 1
print(X_train_b.shape); print(X_test_b.shape)

(1178, 24614)
(785, 24614)


In [21]:
# how to loop over non-zero values
for i, j in zip(*X_train_b.nonzero()):
    print(i, j)
    break

0 360


### Using MLE

Some Naive Bayes notations:  
For each data point $(x,y)$
* $x = [x_1 , x_2 ⋯ x_d]$ and $x_i \in \{0,1\}$
* $y \in \{0,1\}$


Naive bayes assumes that the features are independent given the class label:

* $P(x|y=0)= \Pi \ P(x_i|y=0)$
* $P(x|y=1)= \Pi \ P(x_i|y=1)$

The set of parameters of the classifier are denoted by $θ=\{α, β, γ\}$. The parameters are defined by

* $α=[α_1, α_2, ..., α_d]$
* $α_i=P(x_i=1|y=0, θ)$ $(i \in \{1 \cdots d\})$

---

* $β=[β_1, β_2, ..., β_d]$
* $β_i=P(x_i=1|y=1, θ)$ $(i \in \{1 \cdots d\})$

---

* $γ = P(y=0|θ)$

After using MLE calculation, the following are the parameters:
* Let $n_0$ be the number of data examples with label 0
* Let $n_1$ be the number of data examples with label 1
---
* $\alpha_j = \frac{\underset{i:y^i=0}{\sum} x^i_j}{n_0}$
---
* $\beta_j = \frac{\underset{i:y^i=1}{\sum} x^i_j}{n_1}$
---
* $\gamma = \frac{n_0}{n_0 + n_1}$

In [22]:
# get different sizes
trainSetSize = X_train_b.shape[0]
testSetSize = X_test_b.shape[0]
featureSize = X_train_b.shape[1]

# calculate n0 and n1
class0Index = [] ; class1Index = []
for i in range(trainSetSize):
    if(twenty_train.target[i] == 0):
        class0Index.append(i)
    else:
        class1Index.append(i)
n0 = len(class0Index) ; n1 = len(class1Index)


# get a feature dictionary --> This is essentially a hashmap
# key represents feature index
# value is a list containing document index that contains this feature
documentNonZero = X_train_b.nonzero()[0]
featureNonZero = X_train_b.nonzero()[1]
featureDict = {}
for i in range(featureSize):
    featureDict[i] = []

for i in range(len(featureNonZero)):
    featureDict[featureNonZero[i]].append(documentNonZero[i])

#calculate alpha_i
def getSingleAlphaML(featureIndex):
    sum = 0;
    documents = featureDict[featureIndex]
    for document in documents:
        if (twenty_train.target[document] == 0):
            sum += X_train_b[document , featureIndex]
    return sum / n0


#calculate beta_i
def getSingleBetaML(featureIndex):
    sum = 0;
    documents = featureDict[featureIndex]
    for document in documents:
        if (twenty_train.target[document] == 1):
            sum += X_train_b[document , featureIndex]
    return sum / n1


def estimate_alpha_with_mle():
  # Derive expression of alpha value with MLE.
  # alpha = P(x_i=1|y=0, theta)
    alpha = []
    for i in range(featureSize):
        alpha.append(getSingleAlphaML(i) )
    return alpha

def estimate_beta_with_mle():
  # Derive expression of beta value with MLE
  # beta = P(x_i=1|y=1, theta)

    beta = []
    for i in range(featureSize):
        beta.append(getSingleBetaML(i) )

    return beta

def estimate_gamma_with_mle():
  # Derive expression of gamma value with MLE
  # gamma = P(y=0|theta)
    return n0 / (n0 + n1)


In [23]:
gamma = estimate_gamma_with_mle()
beta = estimate_beta_with_mle()
alpha = estimate_alpha_with_mle()

print(f'gamma is {gamma}')
print(f'beta is {beta[:10]}')
print(f'alpha is {alpha[:10]}')

gamma is 0.49575551782682514
beta is [0.03198653198653199, 0.03198653198653199, 0.0016835016835016834, 0.003367003367003367, 0.0, 0.0, 0.0016835016835016834, 0.0, 0.0, 0.0016835016835016834]
alpha is [0.03424657534246575, 0.018835616438356163, 0.0, 0.0, 0.0017123287671232876, 0.0017123287671232876, 0.0, 0.0017123287671232876, 0.0017123287671232876, 0.0]


Classifier

$p(y=1|x, \theta) = \frac{p(y=1,x,\theta)}{p(x,\theta)} = \frac{p(x|y=1,\theta)*p(y=1|\theta)*p(\theta)}{p(x,\theta)}$

In [24]:
import timeit
import random

class NaiveBayes:
    def __init__(self, gamma, alpha, beta) -> None:
        self.gamma = gamma
        self.alpha = alpha
        self.beta = beta

    def getLabel(self, C1P, C0P):
        if (C1P > C0P):
            return 1
        if (C0P > C1P):
            return 0
        if ( self.gamma > 0.5):
            return 0
        if (( 1 - self.gamma > 0.5)):
            return 1
        return random.choice([0 , 1])


    def predict(self, X):
        dataSetSize = X.shape[0]
        featureSize = X.shape[1]
        predictions = []
        
        # change to numpy array
        X = X.toarray()
        for i in range(dataSetSize):
            class0Prob = self.gamma
            class1Prob = 1 - self.gamma
            for j in range(featureSize):
                if(X[i , j] == 1):
                    class0Prob *= self.alpha[j]
                    class1Prob *= self.beta[j]
                else:
                    class0Prob *= (1 - self.alpha[j])
                    class1Prob *= (1 - self.beta[j])
            label = self.getLabel(class1Prob, class0Prob)
            predictions.append(label)
        return predictions

Testing

In [25]:
classifier = NaiveBayes(gamma, alpha, beta)

TrainSetPredictions = classifier.predict(X_train_b)
TestSetPredictions = classifier.predict(X_test_b)


trainSetSize = X_train_b.shape[0]
correctCount = 0
for i in range(trainSetSize):
    if (TrainSetPredictions[i] == twenty_train.target[i]):
        correctCount += 1
trainSetAccuracy = correctCount / trainSetSize


testSetSize = X_test_b.shape[0]
correctCount = 0
for i in range(testSetSize):
    if(TestSetPredictions[i] == twenty_test.target[i]):
        correctCount += 1
testSetAccuracy = correctCount / testSetSize

print("Accuracy on Training Set(using MLE): " + str(trainSetAccuracy * 100) + " %")
print("Accuracy on Testing  Set(using MLE): " + str(testSetAccuracy * 100) + " %")

Accuracy on Training Set(using MLE): 96.01018675721562 %
Accuracy on Testing  Set(using MLE): 62.54777070063694 %


### Using MAP

* $P(θ) = P(γ).(∏P(α_i)).(∏P(β_i))$
* $P(γ=a)=P(α_1=a)=P(α_2=a)=...=P(β_1=a)=P(β_2=a)=....=f(a)$
    * If $a <= 0.5$ then $f(a) = 4a$
    * If $a > 0.5$ then $f(a) = 4 - 4a$

After calculation:
* If $\alpha_j \leq 0.5$, $\alpha_j = \frac{\left(\underset{i:y^i=0}{\sum} x^i_j \right) + 1}{n_0 + 1}$
---
* If $\alpha_j > 0.5$, $\alpha_j = \frac{\underset{i:y^i=0}{\sum} x^i_j}{n_0 + 1}$
---
* If $\beta_j \leq 0.5$, $\beta_j = \frac{\left( \underset{i:y^i=1}{\sum} x^i_j \right) + 1}{n_1 + 1}$
---
* If $\beta_j > 0.5$, $\beta_j = \frac{ \underset{i:y^i=1}{\sum} x^i_j }{n_1 + 1}$
---
* If $\gamma \leq 0.5$, $\gamma = \frac{n_0 + 1}{n_0 + n_1 + 1}$
---
* If $\gamma > 0.5$, $\gamma = \frac{n_0}{n_0 + n_1 + 1}$


In [26]:
#calculate alpha_i
def getSingleAlphaMAP(featureIndex):
    sum = 0;
    documents = featureDict[featureIndex]
    for document in documents:
        if (twenty_train.target[document] == 0):
            sum += X_train_b[document , featureIndex]
    if ((sum / n0) <= 0.5):
        return (sum + 1) / (n0 +1)
    return sum / (n0 + 1)


#calculate beta_i
def getSingleBetaMAP(featureIndex):
    sum = 0;
    documents = featureDict[featureIndex]
    for document in documents:
        if (twenty_train.target[document] == 1):
            sum += X_train_b[document , featureIndex]
    if ((sum / n1) <= 0.5):
        return (sum + 1) / (n1 + 1)
    return sum / (n1 + 1)


def estimate_alpha_with_map():
  # Derive expression of alpha value with MAP.
  # alpha = P(x_i=1|y=0, theta)
    alpha = []
    for i in range(featureSize):
        alpha.append(getSingleAlphaMAP(i) )
    return alpha


def estimate_beta_with_map():
  # Derive expression of beta value with MAP
  # beta = P(x_i=1|y=1, theta)
    beta = []
    for i in range(featureSize):
        beta.append(getSingleBetaMAP(i) )
    return beta

def estimate_gamma_with_map():
  # Derive expression of gamma value with MAP
  # gamma = P(y=0|theta)
    temp = n0 / (n0 + n1)
    if (temp <= 0.5):
        return (n0 + 1) / (n0 + n1 + 1)
    return n0 / (n0 + n1 + 1)

In [27]:
gamma = estimate_gamma_with_map()
beta = estimate_beta_with_map()
alpha = estimate_alpha_with_map()

print(f'gamma is {gamma}')
print(f'beta is {beta[:10]}')
print(f'alpha is {alpha[:10]}')

gamma is 0.4961832061068702
beta is [0.03361344537815126, 0.03361344537815126, 0.0033613445378151263, 0.005042016806722689, 0.0016806722689075631, 0.0016806722689075631, 0.0033613445378151263, 0.0016806722689075631, 0.0016806722689075631, 0.0033613445378151263]
alpha is [0.035897435897435895, 0.020512820512820513, 0.0017094017094017094, 0.0017094017094017094, 0.003418803418803419, 0.003418803418803419, 0.0017094017094017094, 0.003418803418803419, 0.003418803418803419, 0.0017094017094017094]


In [28]:
classifier = NaiveBayes(gamma, alpha, beta)
TrainSetPredictions = classifier.predict(X_train_b)
TestSetPredictions = classifier.predict(X_test_b)


trainSetSize = X_train_b.shape[0]
correctCount = 0
for i in range(trainSetSize):
    if (TrainSetPredictions[i] == twenty_train.target[i]):
        correctCount += 1
trainSetAccuracy = correctCount / trainSetSize


testSetSize = X_test_b.shape[0]
correctCount = 0
for i in range(testSetSize):
    if(TestSetPredictions[i] == twenty_test.target[i]):
        correctCount += 1
testSetAccuracy = correctCount / testSetSize

print("Accuracy on Training Set(using MAP): " + str(trainSetAccuracy * 100) + " %")
print("Accuracy on Testing  Set(using MAP): " + str(testSetAccuracy * 100) + " %")

Accuracy on Training Set(using MAP): 88.96434634974533 %
Accuracy on Testing  Set(using MAP): 75.54140127388536 %



For accuracy on training set, MLE performs better than MAP. MLE has accuracy of 96.01% on training set and MAP has accuracy of 88.96% on training set.
However, when it comes to testing dataset, MAP(with accuracy of 75.54%)
performed better than MLE(with accuracy of 62.55%). Clearly, MLE method caused overfitting problem.\
\
Justification:
1. For MLE, parameters estimations only base on the dataset we have. It
has no knowledge about the distribution of the parameters. As a result,
MLE cannot generalize the model we are developing so that MLE caused
overfitting problem.
2. For MAP, we can avoid 0 values in $α_i$ and $β_i$. If $α_i$ and $β_i$
calculated from MLE are 0, then 1 will be added to the numerator when
calculating MAP $α_i$ and $β_i$ so that 0 values are avoided in $α_i$ and
$β_i$ for MAP. Therefore, we can avoid 0 probability values when we do the
predictions, which can make predictions more meaningful. As a result,
accuracy improved for testing set.

## SVM Classifier


In [29]:
categories = ['alt.atheism', 'soc.religion.christian',
              'comp.graphics', 'sci.med']
twenty_train = fetch_20newsgroups(subset='train',
    categories=categories, shuffle=True, random_state=42)
twenty_test = fetch_20newsgroups(subset='test',
    categories=categories, shuffle=True, random_state=42)

### Using Linear Kernel

In [30]:
# using CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_test_counts = count_vect.transform(twenty_test.data)

# using TFIDF
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_test_tf = tf_transformer.transform(X_test_counts)

# train SVC
clf = SVC(kernel = 'linear').fit(X_train_tf, twenty_train.target)

# do predictions
trainpreds = clf.predict(X_train_tf)
testpreds = clf.predict(X_test_tf)

def getAccuracy(Yhat , Y):
    dataSize = len(Y)
    correctCount = 0
    for i in range(dataSize):
        if(Yhat[i] == Y[i]):
            correctCount += 1
    return correctCount / dataSize


TrainA = getAccuracy(trainpreds , twenty_train.target)
TestA = getAccuracy(testpreds , twenty_test.target)

print("Accuracy on Training Set(using SVM linear kernel): " + str(TrainA * 100) + " %")
print("Accuracy on Testing  Set(using SVM linaer kernel): " + str(TestA * 100) + " %")

Accuracy on Training Set(using SVM linear kernel): 98.49357554275588 %
Accuracy on Testing  Set(using SVM linaer kernel): 88.08255659121171 %


### Using RBF kernel



In [31]:
clf70 = SVC(kernel = 'rbf' , gamma = 0.70).fit(X_train_tf, twenty_train.target)
clf65 = SVC(kernel = 'rbf' , gamma = 0.65).fit(X_train_tf, twenty_train.target)
clf60 = SVC(kernel = 'rbf' , gamma = 0.60).fit(X_train_tf, twenty_train.target)

trainpreds70 = clf70.predict(X_train_tf)
testpreds70 = clf70.predict(X_test_tf)

trainpreds65 = clf65.predict(X_train_tf)
testpreds65 = clf65.predict(X_test_tf)

trainpreds60 = clf60.predict(X_train_tf)
testpreds60 = clf60.predict(X_test_tf)

TrainA70 = getAccuracy(trainpreds70 , twenty_train.target)
TestA70 = getAccuracy(testpreds70 , twenty_test.target)

TrainA65 = getAccuracy(trainpreds65 , twenty_train.target)
TestA65 = getAccuracy(testpreds65 , twenty_test.target)

TrainA60 = getAccuracy(trainpreds60 , twenty_train.target)
TestA60 = getAccuracy(testpreds60 , twenty_test.target)

print("Accuracy on Training Set(using SVM RBF kernel with gamma = 0.70): " + str(TrainA70 * 100) + " %")
print("Accuracy on Testing  Set(using SVM RBF kernel with gamma = 0.70): " + str(TestA70 * 100) + " %")

print()

print("Accuracy on Training Set(using SVM RBF kernel with gamma = 0.65): " + str(TrainA65 * 100) + " %")
print("Accuracy on Testing  Set(using SVM RBF kernel with gamma = 0.65): " + str(TestA65 * 100) + " %")

print()

print("Accuracy on Training Set(using SVM RBF kernel with gamma = 0.60): " + str(TrainA60 * 100) + " %")
print("Accuracy on Testing  Set(using SVM RBF kernel with gamma = 0.60): " + str(TestA60 * 100) + " %")

Accuracy on Training Set(using SVM RBF kernel with gamma = 0.70): 98.93664155959237 %
Accuracy on Testing  Set(using SVM RBF kernel with gamma = 0.70): 86.21837549933421 %

Accuracy on Training Set(using SVM RBF kernel with gamma = 0.65): 98.84802835622509 %
Accuracy on Testing  Set(using SVM RBF kernel with gamma = 0.65): 86.08521970705726 %

Accuracy on Training Set(using SVM RBF kernel with gamma = 0.60): 98.58218874612317 %
Accuracy on Testing  Set(using SVM RBF kernel with gamma = 0.60): 85.9520639147803 %


Discussion for different gamma values:\
RBF kernel is used to measure the similarity between two points. Smaller gamma value means further influence. As a result, two points can be considered similar even if they are quite far from each other. This is not desirable for classification problem. As a consequence, smaller gamma value produces worse accuracy for classification problem.

### Using IDF

In [32]:
# using CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_test_counts = count_vect.transform(twenty_test.data)

# using TFIDF
tf_transformer = TfidfTransformer(use_idf=True).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_test_tf = tf_transformer.transform(X_test_counts)

clf70 = SVC(kernel = 'rbf' , gamma = 0.70).fit(X_train_tf, twenty_train.target)
clf65 = SVC(kernel = 'rbf' , gamma = 0.65).fit(X_train_tf, twenty_train.target)
clf60 = SVC(kernel = 'rbf' , gamma = 0.60).fit(X_train_tf, twenty_train.target)

trainpreds70 = clf70.predict(X_train_tf)
testpreds70 = clf70.predict(X_test_tf)

trainpreds65 = clf65.predict(X_train_tf)
testpreds65 = clf65.predict(X_test_tf)

trainpreds60 = clf60.predict(X_train_tf)
testpreds60 = clf60.predict(X_test_tf)

TrainA70 = getAccuracy(trainpreds70 , twenty_train.target)
TestA70 = getAccuracy(testpreds70 , twenty_test.target)

TrainA65 = getAccuracy(trainpreds65 , twenty_train.target)
TestA65 = getAccuracy(testpreds65 , twenty_test.target)

TrainA60 = getAccuracy(trainpreds60 , twenty_train.target)
TestA60 = getAccuracy(testpreds60 , twenty_test.target)

print("Accuracy on Training Set(using SVM RBF kernel with gamma = 0.70 idf=True): " + str(TrainA70 * 100) + " %")
print("Accuracy on Testing  Set(using SVM RBF kernel with gamma = 0.70 idf=True): " + str(TestA70 * 100) + " %")

print()

print("Accuracy on Training Set(using SVM RBF kernel with gamma = 0.65 idf=True): " + str(TrainA65 * 100) + " %")
print("Accuracy on Testing  Set(using SVM RBF kernel with gamma = 0.65 idf=True): " + str(TestA65 * 100) + " %")

print()

print("Accuracy on Training Set(using SVM RBF kernel with gamma = 0.60 idf=True): " + str(TrainA60 * 100) + " %")
print("Accuracy on Testing  Set(using SVM RBF kernel with gamma = 0.60 idf=True): " + str(TestA60 * 100) + " %")


Accuracy on Training Set(using SVM RBF kernel with gamma = 0.70 idf=True): 99.9113867966327 %
Accuracy on Testing  Set(using SVM RBF kernel with gamma = 0.70 idf=True): 90.0133155792277 %

Accuracy on Training Set(using SVM RBF kernel with gamma = 0.65 idf=True): 99.9113867966327 %
Accuracy on Testing  Set(using SVM RBF kernel with gamma = 0.65 idf=True): 90.14647137150466 %

Accuracy on Training Set(using SVM RBF kernel with gamma = 0.60 idf=True): 99.9113867966327 %
Accuracy on Testing  Set(using SVM RBF kernel with gamma = 0.60 idf=True): 90.21304926764314 %


Discussion of using idf:\
If we turn on use_idf, the accuracies have increased on both training set
and testing for all three $γ$ values. Turning on use_idf can decrease the
impact of frequent words such as "is", "the", etc. This can increase the impact of other more important features in disguise. As a result, accuracies on both training set and testing set have increased.